In [ ]:
!pip install -r ../requirements.txt

  Using cached tqdm-4.28.1-py2.py3-none-any.whl (45 kB)
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.62.3
    Uninstalling tqdm-4.62.3:
      Successfully uninstalled tqdm-4.62.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spacy 2.2.4 requires tqdm<5.0.0,>=4.38.0, but you have tqdm 4.28.1 which is incompatible.
panel 0.12.1 requires tqdm>=4.48.0, but you have tqdm 4.28.1 which is incompatible.
fbprophet 0.7.1 requires tqdm>=4.36.1, but you have tqdm 4.28.1 which is incompatible.


In [ ]:
model_checkpoint = "t5-small"

In [ ]:
from datasets import load_metric

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/231M [00:00<?, ?B/s]

In [ ]:
import pandas as pd
df = pd.read_csv("../data/train_pairs.csv", index_col=0)
df

In [ ]:
df.reset_index(drop=True, inplace=True)

In [ ]:
df.to_csv("../data/1024_characters_pairs.csv")

In [ ]:
df = df.drop(columns=["title_length", "abstract_length", "token_len"])

In [ ]:
df_train = df[:11864]
df_valid = df[11864:]

In [ ]:
from datasets import Dataset
from datasets import load_dataset, load_metric
train_dataset = Dataset.from_pandas(df_train)
valid_dataset = Dataset.from_pandas(df_valid)
metric = load_metric("rouge")


Downloading:   0%|          | 0.00/2.17k [00:00<?, ?B/s]

In [ ]:
df_train

,title,abstract
0,Natural Image Bases to Represent Neuroimaging ...,Visual inspection of neuroimagery is susceptib...
1,Sluice Resolution without Hand-Crafted Feature...,Sluice resolution in English is the problem of...
2,Sentiment Adaptive End-to-End Dialog Systems,End-to-end learning framework is useful for bu...
3,User-Friendly Text Prediction For Translators,Text prediction is a form of interactive machi...
4,Aligning Sentences from Standard Wikipedia to ...,This work improves monolingual sentence alignm...
...,...,...
11859,Vine Parsing and Minimum Risk Reranking for Sp...,We describe our entry in the CoNLL-X shared ta...
11860,From Characters to Time Intervals: New Paradig...,This paper presents the first model for time n...
11861,Unsupervised Consonant-Vowel Prediction over H...,"In this paper, we present a solution to one as..."
11862,Variational Autoencoder with Arbitrary Conditi...,We propose a single neural probabilistic model...


In [ ]:
max_input_length = 1024
max_target_length = 512

def preprocess_function(examples):
    inputs = ["headline: " + doc for doc in examples["abstract"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["title"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


In [ ]:
preprocess_function(valid_dataset[:2])


{'input_ids': [[12392, 10, 86, 1448, 1254, 1028, 24621, 257, 6, 3, 9, 358, 9048, 12, 2082, 8, 1254, 13, 3, 9, 1448, 45, 28131, 753, 5, 9236, 13121, 12, 740, 3, 9, 306, 18, 27245, 1448, 1254, 1028, 24621, 257, 358, 560, 8, 8565, 13, 3783, 53, 331, 21, 48, 2491, 11, 13, 3, 29856, 1399, 18, 122, 10761, 1254, 13005, 7, 5, 506, 807, 6269, 16509, 45, 8, 685, 24, 8, 2491, 19, 271, 4260, 16, 15997, 45, 487, 2284, 13, 3269, 1028, 24621, 920, 331, 5, 86, 48, 1040, 6, 62, 1099, 8, 1341, 2491, 13, 1448, 7314, 6, 213, 62, 1663, 12, 2082, 8, 2024, 7314, 13, 3, 9, 1448, 45, 2625, 5, 101, 54, 169, 8449, 1612, 11736, 127, 9, 38, 3, 9, 508, 1899, 13, 14610, 3783, 15, 26, 331, 21, 48, 2491, 5, 101, 915, 16783, 21, 11795, 8, 1448, 7314, 682, 11, 5970, 3, 9, 1516, 4179, 147, 3, 9, 20726, 358, 5, 101, 258, 504, 24, 8, 1448, 18, 7031, 6105, 358, 54, 36, 261, 12, 1172, 821, 30, 3, 9, 24687, 1437, 7031, 6105, 2491, 11, 54, 3762, 11, 12700, 27185, 8, 356, 13, 4775, 7314, 7, 21, 3, 9, 1448, 5, 1], [12392, 10, 10

In [ ]:
train_dataset = train_dataset.map(preprocess_function, batched=True)
valid_dataset = valid_dataset.map(preprocess_function, batched=True)


  0%|          | 0/12 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
batch_size = 8
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    "output/T5/" + f"{model_name}-finetuned-lm_al_paper",
    evaluation_strategy = "epoch",
    learning_rate=3e-4,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=False,
    gradient_accumulation_steps=8,
    save_steps = 500,
    logging_steps = 185,
)

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)





```
Examples:
    >>> predictions = ["hello there", "general kenobi"]
    >>> references = ["hello there", "general kenobi"]
    >>> bertscore = datasets.load_metric("bertscore")
    >>> results = bertscore.compute(predictions=predictions, references=references, lang="en")
    >>> print([round(v, 2) for v in results["f1"]])
    [1.0, 1.0]
  
```



In [ ]:
import nltk
import numpy as np
nltk.download('punkt')
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Using amp fp16 backend


In [ ]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: abstract, title.
***** Running training *****
  Num examples = 11864
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 8
  Total optimization steps = 555


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
0,2.806000,2.092597,44.036600,24.640800,39.571800,39.593000,13.657000
1,1.965900,1.956018,45.471500,25.963300,40.534800,40.514300,13.978000
2,1.499900,1.939256,46.978300,26.863500,42.031900,42.036900,14.689000


The following columns in the evaluation set  don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: abstract, title.
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 4
The following columns in the evaluation set  don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: abstract, title.
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 4
Saving model checkpoint to bart-base-finetuned-lm_al_paper/checkpoint-500
Configuration saved in bart-base-finetuned-lm_al_paper/checkpoint-500/config.json
Model weights saved in bart-base-finetuned-lm_al_paper/checkpoint-500/pytorch_model.bin
tokenizer config file saved in bart-base-finetuned-lm_al_paper/checkpoint-500/tokenizer_config.json
Special tokens file saved in bart-base-finetuned-lm_al_paper/checkpoint-500/special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `BartF

TrainOutput(global_step=555, training_loss=2.090629948796453, metrics={'train_runtime': 775.9142, 'train_samples_per_second': 45.871, 'train_steps_per_second': 0.715, 'total_flos': 4491766526607360.0, 'train_loss': 2.090629948796453, 'epoch': 3.0})

In [ ]:
model.save_pretrained("./output/T5")


Configuration saved in ./output/T5/config.json
Model weights saved in ./output/T5/pytorch_model.bin


In [ ]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained("./output/T5")

In [ ]:
import pandas as pd
test_samples = pd.read_csv("path", index_col=0)
test_samples

,title,abstract,title_length,abstract_length
0,Learning Latent Semantic Annotations for Groun...,Previous work on grounded language learning di...,11,121
1,Partially Supervised Sense Disambiguation by L...,Supervised and semi-supervised sense disambigu...,13,140
2,Hawkes Processes for Continuous Time Sequence ...,Classification of temporal textual data sequen...,15,68
3,A Unified Single Scan Algorithm for Japanese B...,We describe an algorithm for Japanese analysis...,13,62
4,Generating Coherent Event Schemas at Scale,Chambers and Jurafsky (2009) demonstrated that...,6,127
...,...,...,...,...
5356,Bridging Information-Seeking Human Gaze and Ma...,"In this work, we analyze how human gaze during...",8,118
5357,Quantum-inspired Neural Network for Conversati...,We provide a novel perspective on conversation...,7,116
5358,The BQ Corpus: A Large-scale Domain-specific C...,This paper introduces the Bank Question (BQ) c...,13,174
5359,Doc2hash: Learning Discrete Latent variables f...,Learning to hash via generative model has beco...,8,131


In [ ]:
abstracts = test_samples.abstract.to_list()
titles = test_samples.title.to_list()

In [ ]:
def creat_eval_pairs(model, tokenizer, abstracts, titles):
  preds = []
  for abstract, title in zip(abstracts, titles):
    encoding = tokenizer.encode_plus("headline: " + abstract, return_tensors = "pt")
    inputs = encoding["input_ids"].to("cuda")
    attention_masks = encoding["attention_mask"].to("cuda")
    title_ids = model.generate(
            input_ids = inputs,
            attention_mask = attention_masks,
            max_length = 30,
            num_beams = 5,
            num_return_sequences = 5,
            repetition_penalty=2.0, 
            length_penalty=10.0,
            early_stopping = True,
            )
    result = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in title_ids]
    s=""
    for t in result:
      s = s + "<TITLE>" + t
    preds.append(s)
    if len(preds) % 500 == 0:
      print("original title: ", title)
      print("generated title: ", preds[-1:])
  return preds, titles

In [ ]:
model.to("cuda")

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseReluDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Dro

In [ ]:
preds, titles = creat_eval_pairs(model, tokenizer, abstracts, titles)

original title:  Paraphrase-Driven Learning for Open Question Answering
generated title:  ['<TITLE>Learning Semantic Lexicon and Linear Ranking without Manual Annotation<TITLE>Learning Semantic Lexicon and Linear Ranking without Manual Annotations<TITLE>Learning Semantic Lexicon and Linear Ranking with Question Answering<TITLE>Learning a Semantic Lexicon and Linear Ranking Function<TITLE>Learning Semantic Lexicon and Linear Ranking Functions']
original title:  Robustness and Generalization of Role Sets: PropBank vs. VerbNet
generated title:  ['<TITLE>Robustness and Generalization of Two Alternative Role Sets for Semantic Role Labeling<TITLE>Robustness and Generalization of Two Alternative Role Sets for Semantic Roles Labeling<TITLE>Robustness and Generalization of Alternative Role Sets for Semantic Role Labeling<TITLE>Robustness and Generalization of Alternative Role Sets for Semantic Roles Labeling<TITLE>Robustness and Generalization of Alternative Role Sets for Semantic Role Labeling

In [ ]:
import pandas as pd
pred_target_pairs = pd.DataFrame(list(zip(preds, titles)), columns=['predictions', 'targets'])

In [ ]:
pred_target_pairs.to_csv("t5-base.csv")